In [62]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# Wikipedia

In [63]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
#URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_Europe'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']

In [46]:
# init dicts
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages[:]:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Skip useless/empty stuff
        if Tags.span == None:
            continue

        # Set chapter variable    
        Chapter = Tags.span.attrs['id']

        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':


            # Get species name
            Species = soup.title\
                            .string.split(' - ')[0]\
                            .rstrip(' ')

            # Create list for key
            DescriptionData[Species] = []
            NonDescriptionData[Species] = []

            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    DescriptionData[Species] += SentenceList

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    NonDescriptionData[Species] += SentenceList

In [27]:
'''
URL = 'https://en.wikipedia.org/wiki/Brown_bear'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for Tags in soup.find_all('h2'):

    # Skip useless/empty stuff
    if Tags.span == None:
        continue

    # Set chapter variable    
    Chapter = Tags.span.attrs['id']

    # Check if the chapter is description (or similar)
    if Chapter == 'Characteristics'or \
       Chapter == 'Description' or \
       Chapter == 'Appearance':


        # Get species name
        Species = soup.title\
                        .string.split(' - ')[0]\
                        .rstrip(' ')

        # Create list for key
        DescriptionData[Species] = []
        NonDescriptionData[Species] = []

        # Get the next sibling (text)
        for Text in Tags.find_next_siblings('p'):

            # Add description data to dict
            if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                DescriptionData[Species] += SentenceList
                
            # Add non description data to dict
            elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                # Remove source
                Paragraph = re.sub(ReferenceRemover, '', Text.text)
                # Split into Sentences
                SentenceList = Paragraph.split('. ')
                # Add to the dict
                NonDescriptionData[Species] += SentenceList
'''

In [50]:
len(DescriptionData['Brown bear'])

52

In [53]:
df_True = pd.DataFrame.from_dict(DescriptionData, orient='index')
df_False = pd.DataFrame.from_dict(NonDescriptionData, orient='index')

In [55]:
df_True.to_pickle("df_true.pkl")
df_False.to_pickle("df_false.pkl")

# Animals A-Z

In [72]:
# init dicts
data = []

# Removes references in text
ReferenceRemover = '\[\d*\]'

URL = 'https://a-z-animals.com/animals/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [83]:
for Tags in soup.find_all('h3'):
    Tags

In [97]:
Animals = soup.find_all('a')

AnimalsA_Zpages = [pages.get('href') for pages in Animals 
                   if pages.get('href') != None
                   if pages.get('href').startswith('https://a-z-animals.com')]

In [98]:
AnimalsA_Zpages

['https://a-z-animals.com/animals/mammals/',
 'https://a-z-animals.com/animals/reptiles/',
 'https://a-z-animals.com/animals/fish/',
 'https://a-z-animals.com/animals/birds/',
 'https://a-z-animals.com/animals/amphibians/',
 'https://a-z-animals.com/pets/',
 'https://a-z-animals.com/pets/cats/',
 'https://a-z-animals.com/pets/dogs/',
 'https://a-z-animals.com/pets/pet-birds/',
 'https://a-z-animals.com/pets/pet-rodents/',
 'https://a-z-animals.com/pets/exotic-pets/',
 'https://a-z-animals.com/pets/fish/',
 'https://a-z-animals.com/reviews/',
 'https://a-z-animals.com/animals/aardvark/',
 'https://a-z-animals.com/animals/abyssinian/',
 'https://a-z-animals.com/animals/adelie-penguin/',
 'https://a-z-animals.com/animals/affenpinscher/',
 'https://a-z-animals.com/animals/afghan-hound/',
 'https://a-z-animals.com/animals/african-bullfrog/',
 'https://a-z-animals.com/animals/african-bush-elephant/',
 'https://a-z-animals.com/animals/african-civet/',
 'https://a-z-animals.com/animals/african